In [1]:
import re 
import os 
#separacao das imagens em listas 
img_reference = 'reference_images_2013'      #caminho das imagens de referencia   
arq_ref = os.listdir(img_reference)     #função para criar uma lista dos arquivos contidos na pasta 

print ('\n')

#manipulando a lista de referencia
new_arq_ref = list ()
for remove in arq_ref:
    remove = remove.rstrip('.BMP')
    new_arq_ref.append(remove)


def converter_maiuscula(lista):
    return [palavra.upper() for palavra in lista]
#lista contendo o nome dos arquivos com as imagens distorcidas
img_distorted = 'distorted_images_2013'
arq_d = os.listdir(img_distorted) # criando uma lista com os arquivos contido na pasta
arq_DIST = converter_maiuscula(arq_d)

new_arq_dist = list ()
for excluir in arq_DIST:
    excluir = excluir.rstrip('.BMP')
    new_arq_dist.append(excluir)


In [ ]:
with open ('mos_with_names_2013.txt','r') as file:
    line = file.readlines()

processed_list = list()

for word in line:
    linha = word.strip()
    processed_list.append(linha)
print (processed_list)

#criando metodo para percorrer a lista 'mos' e a lista 'arq_DIST' para ter certeza que o numero mos faz esta fazendo referencia a imagem deteriorada correta
dividindo_mos = list()
for word in processed_list:
    separar = word.split()
    separar_upper = converter_maiuscula(separar)
    for i in separar_upper:
        if i.startswith('I'):
            dividindo_mos.append(i)
mos_upper = converter_maiuscula(dividindo_mos)
print (mos_upper) #separei a segunda parte da lista mos que contem apenas os diretorio das imagens deterioradas para comparar a ordem de cada uma delas e assim saber que cada numero mesmo esta
                    # correspondendo a mesma imagem deteriorada

count = 0
for ms, dist in zip(mos_upper, arq_DIST):
    count  = count + 1
    if ms == dist:
        print (ms , dist)
    else:
        print ('out of order')
print (count)

In [15]:
#DICIONARIO PARA CALCULAR O SSIM

from ssim import calcular_ssim
import pandas as pd
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_ssim = {}
resultado_ssim = list()
for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_ssim para cada item
                resultado_ssim = calcular_ssim(nova_chave, prefixo_subchave + item + sufixo)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_ssim:
                    dicionario_ssim[chave] = {}
                dicionario_ssim[chave][item] = round(resultado_ssim, 4)
                

print (dicionario_ssim)


{'I01': {'I01_01_1': 0.9593, 'I01_01_2': 0.9268, 'I01_01_3': 0.8722, 'I01_01_4': 0.7966, 'I01_01_5': 0.6945, 'I01_02_1': 0.9885, 'I01_02_2': 0.9784, 'I01_02_3': 0.9587, 'I01_02_4': 0.9266, 'I01_02_5': 0.873, 'I01_03_1': 0.964, 'I01_03_2': 0.9333, 'I01_03_3': 0.8859, 'I01_03_4': 0.8129, 'I01_03_5': 0.717, 'I01_04_1': 0.9821, 'I01_04_2': 0.9651, 'I01_04_3': 0.9335, 'I01_04_4': 0.8844, 'I01_04_5': 0.7938, 'I01_05_1': 0.9581, 'I01_05_2': 0.9246, 'I01_05_3': 0.8234, 'I01_05_4': 0.6545, 'I01_05_5': 0.432, 'I01_06_1': 0.9729, 'I01_06_2': 0.9432, 'I01_06_3': 0.8976, 'I01_06_4': 0.8248, 'I01_06_5': 0.7146, 'I01_07_1': 0.9659, 'I01_07_2': 0.9262, 'I01_07_3': 0.903, 'I01_07_4': 0.8508, 'I01_07_5': 0.7683, 'I01_08_1': 0.9687, 'I01_08_2': 0.8982, 'I01_08_3': 0.7406, 'I01_08_4': 0.5336, 'I01_08_5': 0.3474, 'I01_09_1': 0.953, 'I01_09_2': 0.9063, 'I01_09_3': 0.7934, 'I01_09_4': 0.6304, 'I01_09_5': 0.402, 'I01_10_1': 0.9546, 'I01_10_2': 0.9151, 'I01_10_3': 0.8297, 'I01_10_4': 0.672, 'I01_10_5': 0.5298,

In [ ]:
import pandas as pd 
pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
data_frame = pd.DataFrame(dicionario_ssim)
data_frame.to_excel('SSIM.xlsx')
print(data_frame)



In [16]:
#CALCULANDO O O COEFICIENTE DE SPEARMAN 
from scipy.stats import spearmanr
from scipy.stats import kendalltau
values_ssim = list() #lista de valores da metrica SSIM
for key, value in dicionario_ssim.items():
    for subkey, subvalue in value.items():
        values_ssim.append(subvalue)
print (values_ssim)

spearman_corr, _ = spearmanr(processed_list, values_ssim)
kendall_corr, _ = kendalltau(processed_list, values_ssim)
print ('COEFICIENTE DE SPEARMANR:', spearman_corr)
print ('COEFICIENTE DE KENDALL:', kendall_corr)

[0.9593, 0.9268, 0.8722, 0.7966, 0.6945, 0.9885, 0.9784, 0.9587, 0.9266, 0.873, 0.964, 0.9333, 0.8859, 0.8129, 0.717, 0.9821, 0.9651, 0.9335, 0.8844, 0.7938, 0.9581, 0.9246, 0.8234, 0.6545, 0.432, 0.9729, 0.9432, 0.8976, 0.8248, 0.7146, 0.9659, 0.9262, 0.903, 0.8508, 0.7683, 0.9687, 0.8982, 0.7406, 0.5336, 0.3474, 0.953, 0.9063, 0.7934, 0.6304, 0.402, 0.9546, 0.9151, 0.8297, 0.672, 0.5298, 0.9474, 0.8962, 0.8172, 0.634, 0.3515, 0.9325, 0.9206, 0.815, 0.6786, 0.4932, 0.9586, 0.8931, 0.7964, 0.6933, 0.7051, 0.97, 0.9606, 0.9264, 0.8843, 0.8426, 0.9502, 0.9578, 0.967, 0.9759, 0.9877, 0.9976, 0.9946, 0.9674, 0.963, 0.8535, 0.994, 0.9807, 0.9628, 0.9227, 0.8308, 0.9991, 0.9992, 0.9991, 0.9992, 0.9992, 0.9676, 0.9395, 0.8921, 0.817, 0.7143, 0.9444, 0.898, 0.8172, 0.6592, 0.3935, 0.9468, 0.9025, 0.8287, 0.7115, 0.5502, 0.9502, 0.8994, 0.8339, 0.7842, 0.5374, 0.9935, 0.9789, 0.9568, 0.9179, 0.4357, 0.9433, 0.8855, 0.7768, 0.594, 0.3694, 0.9056, 0.8331, 0.7251, 0.5905, 0.4452, 0.9729, 0.9493, 0

In [ ]:
#DICIONARIO PARA CALCULAR O MSSIM

from MSSIM import calculate_mssim
import pandas as pd
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_mssim = {}
resultado_mssim_2013 = list()

for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_ssim para cada item
                resultado_mssim_2013 = calculate_mssim(nova_chave, prefixo_subchave + item + sufixo)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_mssim:
                    dicionario_mssim[chave] = {}
                dicionario_mssim[chave][item] = round(resultado_mssim_2013[0], 4)
                

print (dicionario_mssim)

In [ ]:
import pandas as pd 

pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
data_frame = pd.DataFrame(dicionario_mssim)
data_frame.to_excel('MSSIM.xlsx')
print(data_frame)


In [33]:
#calculando o coeficiente de spearman:
from scipy.stats import spearmanr
from scipy.stats import kendalltau
mssim_values = list()

for key, value in dicionario_mssim.items():
    for subkey, subvalue in value.items():
        mssim_values.append(subvalue)


spearman_corr_mssim, _ = spearmanr(processed_list, mssim_values)
kendall_corr_mssim, _ = kendalltau(processed_list, mssim_values)
print ('COEFICIENTE DE SPEARMANR:', round(spearman_corr_mssim, 4))
print ('COEFICIENTE DE KENDAL:', round(kendall_corr_mssim, 4))

COEFICIENTE DE SPEARMANR: 0.6389
COEFICIENTE DE KENDAL: 0.4669


In [13]:
#DICIONARIO PARA CALCULAR O VSNR
from vsnr import calcular_vsnr
import pandas as pd
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_VSNR = {}
resultado_VSNR = list()

for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_ssim para cada item
                resultado_VSNR = calcular_vsnr(nova_chave, prefixo_subchave + item + sufixo)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_VSNR:
                    dicionario_VSNR[chave] = {}
                dicionario_VSNR[chave][item] = round(resultado_VSNR, 4)
                

print (dicionario_VSNR)

{'I01': {'I01_01_1': 33.0742, 'I01_01_2': 30.0738, 'I01_01_3': 26.9981, 'I01_01_4': 24.0162, 'I01_01_5': 21.0157, 'I01_02_1': 33.2294, 'I01_02_2': 30.232, 'I01_02_3': 27.204, 'I01_02_4': 24.2479, 'I01_02_5': 21.2308, 'I01_03_1': 33.0919, 'I01_03_2': 30.0514, 'I01_03_3': 26.9866, 'I01_03_4': 24.0043, 'I01_03_5': 21.022, 'I01_04_1': 33.0011, 'I01_04_2': 29.92, 'I01_04_3': 26.847, 'I01_04_4': 24.0874, 'I01_04_5': 20.9545, 'I01_05_1': 32.9979, 'I01_05_2': 29.9976, 'I01_05_3': 25.009, 'I01_05_4': 20.0192, 'I01_05_5': 15.0167, 'I01_06_1': 33.3387, 'I01_06_2': 29.9847, 'I01_06_3': 26.9287, 'I01_06_4': 24.0432, 'I01_06_5': 20.9913, 'I01_07_1': 33.289, 'I01_07_2': 30.2663, 'I01_07_3': 27.128, 'I01_07_4': 24.0697, 'I01_07_5': 21.1297, 'I01_08_1': 34.0126, 'I01_08_2': 29.1857, 'I01_08_3': 25.3369, 'I01_08_4': 22.8195, 'I01_08_5': 21.0833, 'I01_09_1': 32.7027, 'I01_09_2': 30.1412, 'I01_09_3': 26.9414, 'I01_09_4': 24.0491, 'I01_09_5': 21.0453, 'I01_10_1': 33.1252, 'I01_10_2': 30.2636, 'I01_10_3': 2

In [14]:
#CALCULANDO O O COEFICIENTE DE SPEARMAN 
from scipy.stats import spearmanr
from scipy.stats import kendalltau
values_vsnr = list() #lista de valores da metrica SSIM
for key, value in dicionario_VSNR.items():
    for subkey, subvalue in value.items():
        values_vsnr.append(subvalue)

spearman_corr, _ = spearmanr(processed_list, values_vsnr)
kendall_corr, _ = kendalltau(processed_list, values_vsnr)
print ('COEFICIENTE DE SPEARMAN:', round(spearman_corr,3))
print ('COEFICIENTE DE KENDAL:', round(kendall_corr,3))

COEFICIENTE DE SPEARMAN: 0.687
COEFICIENTE DE KENDAL: 0.496


In [ ]:
import pandas as pd 

pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
data_frame = pd.DataFrame(dicionario_VSNR)
data_frame.to_excel('VSNR.xlsx')
print(data_frame)

In [41]:
from mse import calculate_mse
import pandas as pd
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_mse = {}
resultado_mse = list()
for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_mse para cada item
                resultado_mse = calculate_mse(nova_chave, prefixo_subchave + item + sufixo)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_mse:
                    dicionario_mse[chave] = {}
                dicionario_mse[chave][item] = resultado_mse
                

print (dicionario_mse)

{'I01': {'I01_01_1': 14.562403361002604, 'I01_01_2': 27.88933817545573, 'I01_01_3': 47.03214009602865, 'I01_01_4': 64.46768697102864, 'I01_01_5': 78.99293009440105, 'I01_02_1': 3.786224365234375, 'I01_02_2': 7.2902374267578125, 'I01_02_3': 14.707672119140625, 'I01_02_4': 28.064839680989582, 'I01_02_5': 47.05823262532552, 'I01_03_1': 14.435063680013021, 'I01_03_2': 28.273000081380207, 'I01_03_3': 46.357198079427086, 'I01_03_4': 64.55918375651042, 'I01_03_5': 78.76108296712239, 'I01_04_1': 14.635365804036459, 'I01_04_2': 25.691497802734375, 'I01_04_3': 39.797322591145836, 'I01_04_4': 53.16486612955729, 'I01_04_5': 67.28195190429688, 'I01_05_1': 14.711573282877604, 'I01_05_2': 27.978963216145832, 'I01_05_3': 58.06758117675781, 'I01_05_4': 82.48294067382812, 'I01_05_5': 97.7210184733073, 'I01_06_1': 1.1041768391927083, 'I01_06_2': 2.285486857096354, 'I01_06_3': 4.534759521484375, 'I01_06_4': 9.011723836263021, 'I01_06_5': 17.526514689127605, 'I01_07_1': 15.909245808919271, 'I01_07_2': 38.5

In [44]:
#CALCULANDO O O COEFICIENTE DE SPEARMAN 
from scipy.stats import spearmanr
from scipy.stats import kendalltau
from scipy.stats import pearsonr
values_mse = list() #lista de valores da metrica SSIM
for key, value in dicionario_mse.items():
    for subkey, subvalue in value.items():
        values_mse.append(subvalue)

spearman_corr, _ = spearmanr(processed_list, values_mse)
kendall_corr, _ = kendalltau(processed_list, values_mse)
print (spearman_corr)
print (kendall_corr)



-0.4241722643356233
-0.29452846382760844


In [8]:
from FSIMc import calc_fsimc
import pandas as pd
import numpy as np
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_fsimc= {}
resultado_fsimc = list()
for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_mse para cada item
                result_matrix_fsimc = calc_fsimc(nova_chave, prefixo_subchave + item + sufixo)
                # Calcular a média da matriz de similaridade
                resultado_fsimc = np.mean(result_matrix_fsimc)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_fsimc:
                    dicionario_fsimc[chave] = {}
                dicionario_fsimc[chave][item] = resultado_fsimc
                

print (dicionario_fsimc)

{'I01': {'I01_01_1': 0.95451546, 'I01_01_2': 0.9459343, 'I01_01_3': 0.9310699, 'I01_01_4': 0.9127241, 'I01_01_5': 0.8897627, 'I01_02_1': 0.94695705, 'I01_02_2': 0.95557815, 'I01_02_3': 0.95441383, 'I01_02_4': 0.9455602, 'I01_02_5': 0.9314577, 'I01_03_1': 0.94993, 'I01_03_2': 0.9394176, 'I01_03_3': 0.9237613, 'I01_03_4': 0.90290517, 'I01_03_5': 0.8781488, 'I01_04_1': 0.9829418, 'I01_04_2': 0.9785885, 'I01_04_3': 0.970772, 'I01_04_4': 0.9605264, 'I01_04_5': 0.9445905, 'I01_05_1': 0.9621627, 'I01_05_2': 0.9552273, 'I01_05_3': 0.9326213, 'I01_05_4': 0.898498, 'I01_05_5': 0.8533551, 'I01_06_1': 0.9933543, 'I01_06_2': 0.9862769, 'I01_06_3': 0.97401625, 'I01_06_4': 0.9527659, 'I01_06_5': 0.92108274, 'I01_07_1': 0.9257063, 'I01_07_2': 0.9002133, 'I01_07_3': 0.866998, 'I01_07_4': 0.8397598, 'I01_07_5': 0.7784202, 'I01_08_1': 0.971399, 'I01_08_2': 0.9352421, 'I01_08_3': 0.8672519, 'I01_08_4': 0.7840678, 'I01_08_5': 0.7232988, 'I01_09_1': 0.94250304, 'I01_09_2': 0.92051625, 'I01_09_3': 0.87271404

In [ ]:
import pandas as pd 

pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
data_frame = pd.DataFrame(dicionario_fsimc)
data_frame.to_excel('fsimc.xlsx')
print(data_frame)

In [7]:
#CALCULANDO O O COEFICIENTE DE SPEARMAN e KENDALL
from scipy.stats import spearmanr
from scipy.stats import kendalltau
values_FSIMc = list() #lista de valores da metrica SSIM
for key, value in dicionario_fsimc.items():
    for subkey, subvalue in value.items():
        values_FSIMc.append(subvalue)
print (processed_list)

spearman_corr_fsimc, _ = spearmanr(processed_list, values_FSIMc)
kendall_corr_fsimc, _ = kendalltau(processed_list, values_FSIMc)
print ('COEFICIENTE DE SPEARMAN:', spearman_corr_fsimc)
print ('COEFICIENTE DE KENDAL:', kendall_corr_fsimc)

NameError: name 'dicionario_fsimc' is not defined

In [4]:
from PSNR_HA import calc_psnr_ha
import pandas as pd
import numpy as np
arquivosReferencia = new_arq_ref
arquivosDeterioradas = new_arq_dist
listaFinal = {}

for arquivo in arquivosReferencia:
    chaveDoObjeto = arquivo

    # Filtrar imagens deterioradas que pertencem ao arquivo de referência atual
    apenasImagensDeterioradasDaMinhaReferencia = [item for item in arquivosDeterioradas if chaveDoObjeto in item]

    
    listaFinal[arquivo] = {'deterioradas': apenasImagensDeterioradasDaMinhaReferencia}

prefixo = 'reference_images_2013/'  # Substitua 'prefixo_' pelo prefixo desejado
sufixo = '.BMP'
prefixo_subchave = 'distorted_images_2013/'
listaFinalAtualizado = {}

dicionario_psnr_ha= {}
resultado_fsimc = list()
for chave, valor in listaFinal.items():
    nova_chave = prefixo + chave + sufixo
    listaFinalAtualizado[nova_chave] = valor
    
    for subchave, subvalor in valor.items():
        if subchave == 'deterioradas':
            for item in subvalor:
                # Chamar a função calcular_mse para cada item
                resultado_PSNR_HA = calc_psnr_ha(nova_chave, prefixo_subchave + item + sufixo)
                #print( '\n', chave, '\t', item, f'====>  {resultado_ssim:,.3f}')
                if chave not in dicionario_psnr_ha:
                    dicionario_psnr_ha[chave] = {}
                dicionario_psnr_ha[chave][item] = round(resultado_PSNR_HA,4)
                

print (dicionario_psnr_ha)

{'I01': {'I01_01_1': 30.968, 'I01_01_2': 28.7369, 'I01_01_3': 27.1194, 'I01_01_4': 25.8674, 'I01_01_5': 24.5083, 'I01_02_1': 31.4214, 'I01_02_2': 29.2808, 'I01_02_3': 27.6414, 'I01_02_4': 26.252, 'I01_02_5': 24.8644, 'I01_03_1': 31.0371, 'I01_03_2': 28.7066, 'I01_03_3': 27.182, 'I01_03_4': 25.9043, 'I01_03_5': 24.5562, 'I01_04_1': 32.4796, 'I01_04_2': 30.482, 'I01_04_3': 28.9187, 'I01_04_4': 27.6253, 'I01_04_5': 26.0355, 'I01_05_1': 30.9805, 'I01_05_2': 28.8045, 'I01_05_3': 26.4046, 'I01_05_4': 24.0856, 'I01_05_5': 20.829, 'I01_06_1': 51.8096, 'I01_06_2': 48.4457, 'I01_06_3': 45.2713, 'I01_06_4': 42.0051, 'I01_06_5': 38.4703, 'I01_07_1': 1.5814, 'I01_07_2': 1.1434, 'I01_07_3': 0.7628, 'I01_07_4': 0.5082, 'I01_07_5': 0.4077, 'I01_08_1': 32.6619, 'I01_08_2': 29.4224, 'I01_08_3': 27.2084, 'I01_08_4': 25.4877, 'I01_08_5': 24.1445, 'I01_09_1': 31.6652, 'I01_09_2': 30.0289, 'I01_09_3': 28.0295, 'I01_09_4': 25.7269, 'I01_09_5': 21.8056, 'I01_10_1': 32.1639, 'I01_10_2': 30.3404, 'I01_10_3': 28

In [ ]:
import pandas as pd 

pd.set_option('display.max_rows', None)   
pd.set_option('display.max_columns', None) # Mostrar todas as colunas
data_frame = pd.DataFrame(dicionario_psnr_ha)
data_frame.to_excel('psnr_ha.xlsx')
print(data_frame)

In [12]:
#CALCULANDO O O COEFICIENTE DE SPEARMAN e KENDALL
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import os
values_PSNR_HA = list() #lista de valores da metrica SSIM
for key, value in dicionario_psnr_ha.items():
    for subkey, subvalue in value.items():
        values_PSNR_HA.append(subvalue)
print(processed_list)
print (values_PSNR_HA)
spearman_corr_PSNR_HA, _ = spearmanr(processed_list, values_PSNR_HA)
kendall_corr_PSNR_HA, _ = kendalltau(processed_list, values_PSNR_HA)
print ('COEFICIENTE DE SPEARMAN:', spearman_corr_PSNR_HA)
print ('COEFICIENTE DE KENDAL:', kendall_corr_PSNR_HA)

['5.51429 I01_01_1.bmp', '5.56757 i01_01_2.bmp', '4.94444 i01_01_3.bmp', '4.37838 i01_01_4.bmp', '3.86486 i01_01_5.bmp', '6.02857 i01_02_1.bmp', '6.10811 i01_02_2.bmp', '5.38889 i01_02_3.bmp', '4.71429 i01_02_4.bmp', '4.36111 i01_02_5.bmp', '4.77143 i01_03_1.bmp', '4.27027 i01_03_2.bmp', '3.70270 i01_03_3.bmp', '3.19444 i01_03_4.bmp', '2.71429 i01_03_5.bmp', '5.97143 i01_04_1.bmp', '5.75676 i01_04_2.bmp', '5.27027 i01_04_3.bmp', '5.24324 i01_04_4.bmp', '4.71429 i01_04_5.bmp', '5.91892 i01_05_1.bmp', '5.70270 i01_05_2.bmp', '5.08333 i01_05_3.bmp', '4.13889 i01_05_4.bmp', '3.13889 i01_05_5.bmp', '4.00000 i01_06_1.bmp', '3.89189 i01_06_2.bmp', '3.72973 i01_06_3.bmp', '3.48571 i01_06_4.bmp', '2.94444 i01_06_5.bmp', '5.08333 i01_07_1.bmp', '4.75000 i01_07_2.bmp', '4.00000 i01_07_3.bmp', '3.22222 i01_07_4.bmp', '2.19444 i01_07_5.bmp', '5.18919 i01_08_1.bmp', '4.47222 i01_08_2.bmp', '3.48649 i01_08_3.bmp', '3.00000 i01_08_4.bmp', '2.11111 i01_08_5.bmp', '6.08333 i01_09_1.bmp', '5.16667 i01_09